In [1]:
import numpy as np
from astropy.table import Table, join, MaskedColumn, vstack
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from astropy.time import Time
import pandas as pd
import re
import seaborn as sns
import datetime
from datetime import datetime
from datetime import timedelta
from math import e
from math import pi
from astropy.table import Column
from math import sqrt
import numpy as np
import emcee
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import math
import corner
from numpy import exp
from scipy import integrate
from scipy.integrate import quad
import pdb
import powerlaw
import random
from scipy import stats
from multiprocessing import Pool

In [2]:
#Making a model that better represents LEGUS
def err_MCMC(scatter, log_FM):
    sigma, width=scatter
    spread_FM = []
        
    rand_spread= [stats.lognorm.rvs(sigma, 0, width) for i in range(len(log_FM))]

    for i in range(len(log_FM)):
        rand_from_norm=(np.random.normal(0, rand_spread[i]))
        spread_FM.append(log_FM[i]+rand_from_norm)
        
    spread_FM=np.array(spread_FM)
    spread_FM
    cut=np.where(spread_FM<(np.log10(1086)))
    cut_spread=np.delete(spread_FM, cut)
        
        
    """"Running MCMC on new distribution"""
    starting_point=np.array([-2.00, 4.00])
    ndim, nwalkers = 2, 500
    nsteps= 600
    burnin=100
    pos = starting_point + 1e-2*np.random.randn(nwalkers, ndim)
        

    e_sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, pool=pool, args=([cut_spread]))
    e_sampler.run_mcmc(pos, nsteps)
        
    e_sampler.get_chain()
    e_samples = e_sampler.chain[:, burnin:, :].reshape((-1, ndim))
    
    alpha=[i[0] for i in e_samples]
    Mc= [i[1] for i in e_samples]
        
    med_a=np.median(alpha)
    med_Mc=np.median(Mc)

        
    return np.array((med_a, med_Mc))

def lnZ(theta, M):
    alpha, M_c = theta
    lin_M_c= 10**M_c
    def f(M):
        return (M**alpha)*exp(-M/lin_M_c)
    ans, err = quad(f, 5000, np.inf)
    return np.log(ans)

def lnlike(theta, M):
    alpha, M_c = theta
    lin_M= 10**M
    lin_M_c= 10**M_c
    return (np.sum(-lin_M/lin_M_c + alpha*np.log(lin_M) - lnZ(theta, lin_M)))

def lnprior(theta):
    alpha, M_c = theta
    if -3 <= alpha <= -1 and 3 <= M_c <= 8:
        return 0.0
    return -np.inf

def lnprob(theta, M):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, M)


def error_calc_lognorm_LEGUS():
    """"Creating Fake data set"""
    
    theoretical_distribution = powerlaw.Power_Law(xmin=1000, xmax=100000, parameters = [2], discrete=True)
    simulated_data=theoretical_distribution.generate_random(5200)
    fake_M_l=[]
    for i in range(len(simulated_data)):
        fake_M_l.append(simulated_data[i])

    A3_fml=[]
    for i in range(len(fake_M_l)):
        if fake_M_l[i] >=1000 and fake_M_l[i] < 10**6.2:
            A3_fml.append(fake_M_l[i])
        
    A3_fml.sort()

    fake_M=np.array(A3_fml)
    fake_M

#    print(np.where(fake_M>100000))

    random_ints = np.array(random.sample(range(5150, 5200),35))
#    random_ints2 = np.array(random.sample(range(30, 2900), 700))
    new_fake_M=np.delete(fake_M, [random_ints])
#    new_fake_M2=np.delete(new_fake_M, [random_ints2])

    log_FMl=[]
    for i in range(len(new_fake_M)):
        log_FMl.append(np.log10(new_fake_M[i]))
    
    log_FM= np.array(log_FMl)
    log_FM
    
    cut=np.where(log_FM<(np.log10(1086)))
    cut_FM=np.delete(log_FM, cut)

    """"Running MCMC"""
    starting_point=np.array([-2.00, 4.00])

    ndim, nwalkers = 2, 500
    nsteps= 600
    burnin=100
    pos = starting_point + 1e-2*np.random.randn(nwalkers, ndim)

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, pool=pool, args=([cut_FM]))
    sampler.run_mcmc(pos, nsteps)

    sampler.get_chain()
    samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
    
    true_alpha=[i[0] for i in samples]
    true_Mc= [i[1] for i in samples]
    med_ta=np.median(true_alpha)
    med_tMc= np.median(true_Mc)

    distribution=np.array((med_ta, med_tMc))
    
    
    """"Generating Scatter in the data"""
    
    
    scatter=[(.5, .05), (.75, .05), (1.0, .05), (1.25, .05), 
             (.5, .1), (.75, .1), (1.0, .1), (1.25, .1),  
             (.5, .15), (.75, .15), (1.0, .15), (1.25, .15), 
             (.5, .2), (.75, .2), (1.0, .2), (1.25, .2)]
        
    results=np.zeros((len(scatter)+1, 2))    
    
    results[0]= distribution
    
    for i in range(1,len(scatter)+1):
        for k in range(len(scatter)):
            results[i]=err_MCMC(scatter[(i-1)], log_FM)
        
    return results


In [3]:
def plot_ln(first_round):
    true=first_round[0]
    point05=first_round[1:5]
    point1=first_round[5:9]
    point15=first_round[9:13]
    point2=first_round[13:17]
    
    p05a0=[i[0] for i in point05]
    p05M0=[i[1] for i in point05]
    p1a0=[i[0] for i in point1]
    p1M0=[i[1] for i in point1]
    p15a0=[i[0] for i in point15]
    p15M0=[i[1] for i in point15]
    p2a0=[i[0] for i in point2]
    p2M0=[i[1] for i in point2]
    
    sig5=first_round[np.array([1,5,9,13])]
    sig75=first_round[np.array([2,6,10,14])]
    sig1=first_round[np.array([3,7,11,15])]
    sig125=first_round[np.array([4,8,12,16])]
    
    sig5a0=[i[0] for i in sig5]
    sig5M0=[i[1] for i in sig5]
    sig75a0=[i[0] for i in sig75]
    sig75M0=[i[1] for i in sig75]
    sig1a0=[i[0] for i in sig1]
    sig1M0=[i[1] for i in sig1]
    sig125a0=[i[0] for i in sig125]
    sig125M0=[i[1] for i in sig125]
    

    p05a_dist_from_true=[]
    for i in range(len(p05a0)):
        p05a_dist_from_true.append(p05a0[i]- true[0])
    
    p1a_dist_from_true=[]
    for i in range(len(p1a0)):
        p1a_dist_from_true.append(p1a0[i]- true[0])
        
    p15a_dist_from_true=[]
    for i in range(len(p15a0)):
        p15a_dist_from_true.append(p05a0[i]- true[0])
        
    p2a_dist_from_true=[]
    for i in range(len(p2a0)):
        p2a_dist_from_true.append(p2a0[i]- true[0])

    p05Mc_dist_from_true=[]
    for i in range(len(p05a0)):
        p05Mc_dist_from_true.append(p05M0[i]- true[1])
        
    p1Mc_dist_from_true=[]
    for i in range(len(p1a0)):
        p1Mc_dist_from_true.append(p1M0[i]- true[1])
        
    p15Mc_dist_from_true=[]
    for i in range(len(p15a0)):
        p15Mc_dist_from_true.append(p15M0[i]- true[1])
        
    p2Mc_dist_from_true=[]
    for i in range(len(p2a0)):
        p2Mc_dist_from_true.append(p2M0[i]- true[1])
        
        
    sig5a_dist_from_true=[]
    for i in range(len(sig5a0)):
        sig5a_dist_from_true.append(sig5a0[i]- true[0])
    
    sig75a_dist_from_true=[]
    for i in range(len(sig75a0)):
        sig75a_dist_from_true.append(sig75a0[i]- true[0])
        
    sig1a_dist_from_true=[]
    for i in range(len(sig1a0)):
        sig1a_dist_from_true.append(sig1a0[i]- true[0])
        
    sig125a_dist_from_true=[]
    for i in range(len(sig125a0)):
        sig125a_dist_from_true.append(sig125a0[i]- true[0])

    sig5Mc_dist_from_true=[]
    for i in range(len(sig5a0)):
        sig5Mc_dist_from_true.append(sig5M0[i]- true[1])
        
    sig75Mc_dist_from_true=[]
    for i in range(len(sig75a0)):
        sig75Mc_dist_from_true.append(sig75M0[i]- true[1])
        
    sig1Mc_dist_from_true=[]
    for i in range(len(sig1a0)):
        sig1Mc_dist_from_true.append(sig1M0[i]- true[1])
        
    sig125Mc_dist_from_true=[]
    for i in range(len(sig125a0)):
        sig125Mc_dist_from_true.append(sig125M0[i]- true[1])
    
    
    scatter=[.5, .75, 1, 1.25]
    
    mu=[.05,.1,.15,.2]
    
    ax1=plt.subplot(2,4,1)
    ax1.scatter(mu, sig5Mc_dist_from_true, color='r', marker='o', label='Sigma=.5: Mc')
    ax1.set_xlabel('e^(Mu) for Error Distribition')
    ax1.set_ylabel('Distance from True Value')
    ax1.legend(loc=2)
    
    ax2=plt.subplot(2,4,2, sharex=ax1, sharey=ax1)
    ax2.scatter(mu, sig75Mc_dist_from_true, color='b', marker='*', label='Sigma=.75: Mc')
    ax2.set_xlabel('e^(Mu) for Error Distribition')
#    ax2.set_ylabel('Distance from True Value')
    ax2.legend(loc=2)
    
    ax3=plt.subplot(2,4,3, sharex=ax1, sharey=ax1)
    ax3.scatter(mu, sig1Mc_dist_from_true, color='g', marker='+', label='Sigma=1.0: Mc', s=69)
    ax3.set_xlabel('e^(Mu) for Error Distribition')
#    ax3.set_ylabel('Distance from True Value')
    ax3.legend()
    
    ax4=plt.subplot(2,4,4, sharex=ax1, sharey=ax1)
    ax4.scatter(mu, sig125Mc_dist_from_true, color='y', marker='D', label='Sigma=1.25: Mc',s=20)
    ax4.set_xlabel('e^(Mu) for Error Distribition')
#    ax4.set_ylabel('Distance from True Value')
    ax4.legend(loc=2)
    
    ax5=plt.subplot(2,4,5)
    ax5.scatter(mu, sig5a_dist_from_true, color='r', marker='o', label='Sigma=.5: Alpha')
    ax5.set_xlabel('e^(Mu) for Error Distribition')
    ax5.set_ylabel('Distance from True Value')
    ax5.legend(loc=2)
    
    ax6=plt.subplot(2,4,6, sharex=ax5, sharey=ax5)
    ax6.scatter(mu, sig75a_dist_from_true, color='b', marker='*', label='Sigma=.75: Alpha')
    ax6.set_xlabel('e^(Mu) for Error Distribition')
#    ax6.set_ylabel('Distance from True Value')
    ax6.legend(loc=2)
    
    ax7=plt.subplot(2,4,7, sharex=ax5, sharey=ax5)
    ax7.scatter(mu, sig1a_dist_from_true, color='g', marker='+', label='Sigma=1.0: Alpha', s=69)
    ax7.set_xlabel('e^(Mu) for Error Distribition')
#    ax7.set_ylabel('Distance from True Value')
    ax7.legend(loc=2)
    
    ax8=plt.subplot(2,4,8, sharex=ax5, sharey=ax5)
    ax8.scatter(mu, sig125a_dist_from_true, color='y', marker='D', label='Sigma=1.25: Alpha', s=20)
    ax8.set_xlabel('e^(Mu) for Error Distribition')
#    ax8.set_ylabel('Distance from True Value')
    ax8.legend(loc=2)
    
    plt.subplots_adjust(right=2.2, top=1.5)
    plt.show()
    
    return np.array([sig5a_dist_from_true, sig5Mc_dist_from_true, sig75a_dist_from_true, sig75Mc_dist_from_true, 
                     sig1a_dist_from_true, sig1Mc_dist_from_true, sig125a_dist_from_true, sig125Mc_dist_from_true]) 

